In [1]:
#!/usr/bin/env python
# coding: utf-8

import os
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.prompts import PromptTemplate#
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQAWithSourcesChain, RetrievalQA
from langchain import OpenAI
import gradio as gr

In [2]:

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_TOKEN')

embeddings = OpenAIEmbeddings()
db = Chroma(persist_directory="db", embedding_function=embeddings)


Using embedded DuckDB with persistence: data will be stored in: db


In [3]:

prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer in German:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [4]:

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(),
                                 chain_type="stuff",
                                 retriever=db.as_retriever(),
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)


In [11]:
docs = db.similarity_search(query)
qa_rerank = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_rerank", metadata_keys=['source'], return_intermediate_steps=True)


In [13]:
query = "Wann hat Prof. Dr. Wildemann einen Verdiensorden erhalten?"
result = qa_rerank({"input_documents": docs, "question": query}, return_only_outputs=True)

In [23]:
from langchain.output_parsers import RegexParser

output_parser = RegexParser(
    regex=r"(.*?)\nScore: (.*)",
    output_keys=["answer", "score"],
)

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. 
If the answer contains of multiple parts, give it in bullet points. 

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer In German: [answer here]
Score: [score between 0 and 100]

Begin!

Context:
---------
{context}
---------
Question: {question}
Helpful Answer In German:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
    output_parser=output_parser,
)
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_rerank", metadata_keys=['source'], return_intermediate_steps=True, prompt=PROMPT)
query = "Welche Dienstleistungen bietet TCW an?"
result = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
result


{'source': 'scraper/html/tcw.de/tcw.de_sitemap.html',
 'intermediate_steps': [{'answer': ' Ich weiß es nicht.', 'score': '0'},
  {'answer': ' TCW bietet Beratungsdienstleistungen, Finanzdienstleistungen und Immobiliendienstleistungen an.',
   'score': '100'},
  {'answer': ' TCW bietet Unternehmensberatung an.', 'score': '100'},
  {'answer': ' TCW bietet Beratungsdienstleistungen in den Bereichen Finanzen, Steuern, Recht und Unternehmensführung an.',
   'score': '100'}],
 'output_text': ' TCW bietet Beratungsdienstleistungen, Finanzdienstleistungen und Immobiliendienstleistungen an.'}

In [71]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
In case the question has multiple answers, formulate the answer in bullet points. 

{context}

Question: {question}
Answer in German:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
) 

chain_type_kwargs = {"prompt": PROMPT}

def get_chat_history(inputs) -> str:
    res = []
    for human, ai in inputs:
        res.append(f"Human: {human}\nAI: {ai}")
    return "\n".join(res)

chat_history = []
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa = ConversationalRetrievalChain.from_llm(llm = OpenAI(temperature=0),
                                           retriever = db.as_retriever(),
                                           qa_prompt=PROMPT,
                                           chain_type="stuff", 
                                           return_source_documents = False, 
                                           memory = memory, 
                                           #get_chat_history=get_chat_history, 
                                           verbose = True)


In [96]:
memory.chat_memory.messages

[HumanMessage(content='Wer ist Prof. Dr. Wildemann?', additional_kwargs={}),
 AIMessage(content=' \nProf. Dr. Wildemann ist ein ordentlicher Professor für Betriebswirtschaftslehre an den Universitäten Bayreuth, Passau und seit 1989 an der Technischen Universität München. Er hat Maschinenbau (Dipl.-Ing.) und Betriebswirtschaftslehre (Dipl.-Kfm.) an der Universität Aachen und Köln studiert und promovierte 1974 zum Dr. rer. pol. Er habilitierte sich 1980 an der Universität zu Köln (Dr. habil.).', additional_kwargs={}),
 HumanMessage(content='Hat er Auszeichnungen erhalten?', additional_kwargs={}),
 AIMessage(content=' \n- Staatsmedaille des Freistaates Bayern \n- Bundesverdienstkreuz 1. Klasse der Bundesrepublik Deutschland \n- Ehrendoktorwürden der Universitäten Klagenfurt, Passau und Cottbus \n- Logistik Hall of Fame \n- Bayerischen Verdienstorden \n- Ehrennadel der Bundesvereinigung Logistik', additional_kwargs={})]

In [72]:
query = "Wer ist Prof. Dr. Wildemann?"
result = qa({"question": query, "chat_history" : chat_history})
#chat_history.append((query, result["answer"]))
result



> Entering new ConversationalRetrievalChain chain...

> Finished chain.


{'question': 'Wer ist Prof. Dr. Wildemann?',
 'chat_history': [HumanMessage(content='Wer ist Prof. Dr. Wildemann?', additional_kwargs={}),
  AIMessage(content=' \nProf. Dr. Wildemann ist ein ordentlicher Professor für Betriebswirtschaftslehre an den Universitäten Bayreuth, Passau und seit 1989 an der Technischen Universität München. Er hat Maschinenbau (Dipl.-Ing.) und Betriebswirtschaftslehre (Dipl.-Kfm.) an der Universität Aachen und Köln studiert und promovierte 1974 zum Dr. rer. pol. Er habilitierte sich 1980 an der Universität zu Köln (Dr. habil.).', additional_kwargs={})],
 'answer': ' \nProf. Dr. Wildemann ist ein ordentlicher Professor für Betriebswirtschaftslehre an den Universitäten Bayreuth, Passau und seit 1989 an der Technischen Universität München. Er hat Maschinenbau (Dipl.-Ing.) und Betriebswirtschaftslehre (Dipl.-Kfm.) an der Universität Aachen und Köln studiert und promovierte 1974 zum Dr. rer. pol. Er habilitierte sich 1980 an der Universität zu Köln (Dr. habil.).'}

In [73]:
query = "Hat er Auszeichnungen erhalten?"
result = qa({"question": query, "chat_history" : chat_history})
#chat_history.append((query, result["answer"]))
result



> Entering new ConversationalRetrievalChain chain...

> Finished chain.


{'question': 'Hat er Auszeichnungen erhalten?',
 'chat_history': [HumanMessage(content='Wer ist Prof. Dr. Wildemann?', additional_kwargs={}),
  AIMessage(content=' \nProf. Dr. Wildemann ist ein ordentlicher Professor für Betriebswirtschaftslehre an den Universitäten Bayreuth, Passau und seit 1989 an der Technischen Universität München. Er hat Maschinenbau (Dipl.-Ing.) und Betriebswirtschaftslehre (Dipl.-Kfm.) an der Universität Aachen und Köln studiert und promovierte 1974 zum Dr. rer. pol. Er habilitierte sich 1980 an der Universität zu Köln (Dr. habil.).', additional_kwargs={}),
  HumanMessage(content='Hat er Auszeichnungen erhalten?', additional_kwargs={}),
  AIMessage(content=' \n- Staatsmedaille des Freistaates Bayern \n- Bundesverdienstkreuz 1. Klasse der Bundesrepublik Deutschland \n- Ehrendoktorwürden der Universitäten Klagenfurt, Passau und Cottbus \n- Logistik Hall of Fame \n- Bayerischen Verdienstorden \n- Ehrennadel der Bundesvereinigung Logistik', additional_kwargs={})],
 

In [83]:
result["chat_history"]

[HumanMessage(content='Wer ist Prof. Dr. Wildemann?', additional_kwargs={}),
 AIMessage(content=' \nProf. Dr. Wildemann ist ein ordentlicher Professor für Betriebswirtschaftslehre an den Universitäten Bayreuth, Passau und seit 1989 an der Technischen Universität München. Er hat Maschinenbau (Dipl.-Ing.) und Betriebswirtschaftslehre (Dipl.-Kfm.) an der Universität Aachen und Köln studiert und promovierte 1974 zum Dr. rer. pol. Er habilitierte sich 1980 an der Universität zu Köln (Dr. habil.).', additional_kwargs={}),
 HumanMessage(content='Hat er Auszeichnungen erhalten?', additional_kwargs={}),
 AIMessage(content=' \n- Staatsmedaille des Freistaates Bayern \n- Bundesverdienstkreuz 1. Klasse der Bundesrepublik Deutschland \n- Ehrendoktorwürden der Universitäten Klagenfurt, Passau und Cottbus \n- Logistik Hall of Fame \n- Bayerischen Verdienstorden \n- Ehrennadel der Bundesvereinigung Logistik', additional_kwargs={})]

In [52]:
query = "Über wen reden wir?"
result = qa({"question": query, "chat_history" : chat_history})
#chat_history.append((query, result["answer"]))
result



> Entering new ConversationalRetrievalChain chain...

> Finished chain.


{'question': 'Über wen reden wir?',
 'chat_history': [],
 'answer': ' Wir reden über Mehr Infos zum Thema.'}

In [26]:
pruiget_chat_history(chat_history)

'Human: Wer ist Prof. Dr. Wildemann?\nAI:  \nProf. Dr. Wildemann ist ein ordentlicher Professor für Betriebswirtschaftslehre an den Universitäten Bayreuth, Passau und seit 1989 an der Technischen Universität München. Er hat Maschinenbau (Dipl.-Ing.) und Betriebswirtschaftslehre (Dipl.-Kfm.) an der Universität Aachen und Köln studiert und promovierte 1974 zum Dr. rer. pol. Er habilitierte sich 1980 an der Universität zu Köln (Dr. habil.).'

In [22]:
chat_history

[[('Wer ist Prof. Dr. Wildemann?',
   ' \nProf. Dr. Wildemann ist ein ordentlicher Professor für Betriebswirtschaftslehre an den Universitäten Bayreuth, Passau und seit 1989 an der Technischen Universität München. Er hat Maschinenbau (Dipl.-Ing.) und Betriebswirtschaftslehre (Dipl.-Kfm.) an der Universität Aachen und Köln studiert und promovierte 1974 zum Dr. rer. pol. Er habilitierte sich 1980 an der Universität zu Köln (Dr. habil.).')]]

In [15]:
print(result["chat_history"][0])

Human: Wer ist Prof. Dr. Wildemann?
AI:  
Prof. Dr. Wildemann ist ein ordentlicher Professor für Betriebswirtschaftslehre an den Universitäten Bayreuth, Passau und seit 1989 an der Technischen Universität München. Er hat Maschinenbau (Dipl.-Ing.) und Betriebswirtschaftslehre (Dipl.-Kfm.) an der Universität Aachen und Köln studiert und promovierte 1974 zum Dr. rer. pol. Er habilitierte sich 1980 an der Universität zu Köln (Dr. habil.).


In [ ]:

def qa_query(query):
    result = qa({"query": query})
    return result

with gr.Blocks() as tcw_bot:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Enter question and press enter", show_label=False)
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]
    
    def bot(history):
        response = qa_query(history[-1][0])
        print(response)
        response_result = response["result"]
        metadata = response["source_documents"]
        history[-1][1] = response_result

        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)


tcw_bot.launch(share=False)